In [41]:
import skrf as rf
from skrf.constants import K_BOLTZMANN
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from scipy.interpolate import interp1d
from devices.devices_lib import make_cavity__with_antennas_network # From Chirstan Boutan's PNNL team
import components as cmp # From Jihee
import os.path

%matplotlib inline

In [18]:
freq = rf.frequency.Frequency(start=1, stop=3, npoints=1001, unit='ghz', sweep_type='lin')

In [1]:
###############################
##### Component Functions   ###
###############################
#(From Jihee's components.py and analyis_run1C_tsys.py)


### Function to read network in noisy environment ###
def read_in_noisy_network(ntw,freq_interp,Tphys):
    ntw_interp = ntw.interpolate(freq_interp, kind='cubic')
    ntw_interp_noisy = rf.NoisyNetwork(ntw_interp)
    ntw_interp_noisy.noise_source(source='passive',T0 = Tphys)

    return ntw_interp_noisy

### Function to make coaxial cable with coaxial line and SMA connector sample ###
def make_coax_cable(c_type,freq,len_m,Tphys):
        
    if c_type=='RG405U':
        ntw_coax_RT = cmp.cable_RG405U(freq,len_m)
        ntw_coax_cryo = cmp.RT2Cryo(ntw_coax_RT)
        # ntw_coax_cryo = ntw_coax_RT
        # ntw_SMA_RT = cmp.cable_RG405U(freq,0.04)
        ntw_SMA_RT = cmp.SMA_RG405U(freq,0.015)
        ntw_SMA_cryo = cmp.RT2Cryo(ntw_SMA_RT)
        # ntw_SMA_cryo = ntw_SMA_RT
    elif c_type=='PESR405FL':    
        ntw_coax_RT = cmp.cable_PESR405FL(freq,len_m)
        ntw_coax_cryo = cmp.RT2Cryo(ntw_coax_RT)
        # ntw_coax_cryo = ntw_coax_RT
        # ntw_SMA_RT = cmp.cable_PESR405FL(freq,0.04)
        ntw_SMA_RT = cmp.SMA_PESR405FL(freq,0.015)
        ntw_SMA_cryo = cmp.RT2Cryo(ntw_SMA_RT)
        # ntw_SMA_cryo = ntw_SMA_RT
    elif c_type=='NbTi085':
        ntw_coax_cryo = cmp.cable_NbTi085(freq,len_m)
        ntw_SMA_cryo = cmp.cable_NbTi085(freq,0.015)
    else:
        print('Error: Proper cable type is missing')

    noisy_ntw_coax = read_in_noisy_network(ntw_coax_cryo,freq,Tphys)
    noisy_ntw_SMA = read_in_noisy_network(ntw_SMA_cryo, freq, Tphys)
    
    assmb_cable = rf.MultiNoisyNetworkSystem()

    noisy_ntw_SMA.add_noise_polar(1e-6, 0.5)
    assmb_cable.add(noisy_ntw_SMA,'SMA1')
    assmb_cable.add(noisy_ntw_coax,'coax')
    noisy_ntw_SMA.add_noise_polar(1e-6, 0.55)
    assmb_cable.add(noisy_ntw_SMA,'SMA2')

    assmb_cable.connect('SMA1',2, 'coax',1)
    assmb_cable.connect('coax',2, 'SMA2',1)

    assmb_cable.external_port('SMA1',1,1)
    assmb_cable.external_port('SMA2',2,2)

    noisy_ntw_cable = assmb_cable.reduce()
    noisy_ntw_cable = read_in_noisy_network(noisy_ntw_cable,freq,Tphys)
        
    return noisy_ntw_cable  

In [64]:
def gen_ZX60_83LN_S_NF_func(ZX60_83LN_NF_loc):
    noise_figure_data2 = pd.read_csv(ZX60_83LN_NF_loc , header = None, encoding='unicode_escape')
    noise_figure_data2[11] = 10**(noise_figure_data2[10]/10)
    NF_Func2 = interp1d(noise_figure_data2[0],noise_figure_data2[11],fill_value=1000)
    
    return NF_Func2

def return_ZX60_83LN_params(Amp_path,Amp_name,freq,sat_check=False,vendor=True,Temp=290):
    
    # ZX60_83LN_fpath = r'.\raw data\raw data'
    # ZX60_83LN_noise_figure_fnmae = r'ZX60_83LN_NF.csv'
    # ZX60_83LN_NF_loc = os.path.join(ZX60_83LN_fpath, ZX60_83LN_noise_figure_fname)
    ZX60_83LN_NF_loc = r'/Users/zacharybanken/Desktop/research_stuff/Terminator Temp Test/amplifier_data/ZX60_83LN_NF.csv'
    
    Tphys = Temp
    
    Amp_loc = os.path.join(Amp_path,Amp_name)
    amp_sparam = rf.Network(Amp_loc)
    amp_sparam = amp_sparam.interpolate(freq,kind='cubic')
    amp_sparam  = rf.NoisyNetwork(amp_sparam)
    amp_sparam.noise_source(source='passive',T0=Tphys)
    
    try:
        NF_func = gen_ZX60_83LN_S_NF_func(ZX60_83LN_NF_Loc)
        nfig = NF_func(freq.f/1e9)
    except:
        print('error')
        
    Tnoise = rf.NetworkNoiseCov.Tnoise(freq.f,Tphys)
    
    I = np.identity(np.shape(amp_sparam.s)[1])
    AS = np.matmul(I,np.conjugate(amp_sparam.s.swapaxes(1,2)))
    SAS = np.matmul(amp_sparam.s,AS)
    
    amp_sparam.cs[:,0,0] = (nfig - 1) * K_BOLTZMANN * 290 * SAS[:,0,0]
    
    I = np.identity(np.shape(amp_sparam.s)[1])
    AS = np.matmul(I,np.conjugate(amp_sparam.s.swapaxes(1,2)))
    SAS = np.matmul(amp_sparam.s,AS)
    
    amp_sparam.cs[:,1,1] = (nfig-1) * K_BOLTZMANN * 290 * SAS[:,1,1]
    
    amp_sparam.cs[:,0,1] = np.zeros_like(amp_sparam.cs[:,1,1])
    amp_sparam.cs[:,1,0] = amp_sparam.cs[:,0,1]
    
    return amp_sparam
    

In [69]:
amp_sparam = rf.Network(r'/Users/zacharybanken/Desktop/research_stuff/Terminator Temp Test/amplifier_data/ZX60-83LN-S+_AP160503_060216_RF PARAM_Unit2.s2p')







UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 127: invalid start byte

In [74]:
cav = rf.Network('/Users/zacharybanken/Desktop/research_stuff/Terminator\ Temp\ Test/amplifier_data/ZX60-83LN-S+_AP160503_060216_RF PARAM_Unit2.s2p')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 127: invalid start byte